## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...
1,2026-01-11-13-00-00 +0000,wapo,"With YouTube and TikTok, dentists are trying t...",https://www.washingtonpost.com/nation/2026/01/...
2,2026-01-11-12-26-18 +0000,wapo,‘Massacre’ feared as Iran tries to crush prote...,https://www.washingtonpost.com/world/2026/01/1...
3,2026-01-11-12-20-57 +0000,bbc,Potholes map rates council road repair progress,https://www.bbc.com/news/articles/ckgj1m1ze8go...
4,2026-01-11-12-07-16 +0000,bbc,"Tories will channel anger at Labour, vows Bade...",https://www.bbc.com/news/articles/c0jvz020zzyo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,35
102,ice,22
22,iran,17
160,venezuela,17
362,minneapolis,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2026-01-11-10-35-00 +0000,wsj,Iran Warns It Could Hit U.S. Bases as Trump We...,https://www.wsj.com/world/middle-east/iran-war...,76
130,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,73
110,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...,72
145,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...,72
169,2026-01-10-15-19-51 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,76,2026-01-11-10-35-00 +0000,wsj,Iran Warns It Could Hit U.S. Bases as Trump We...,https://www.wsj.com/world/middle-east/iran-war...
110,72,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...
49,47,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...
131,38,2026-01-10-20-08-00 +0000,wsj,Maduro’s Crypto-Backed Oil Deals Put Tether at...,https://www.wsj.com/finance/currencies/maduros...
148,31,2026-01-10-17-47-13 +0000,nypost,New NYC Council GOP leader ready to battle Mam...,https://nypost.com/2026/01/10/us-news/new-nyc-...
137,30,2026-01-10-18-52-27 +0000,nypost,"Ilhan Omar squares off against feds, joins Min...",https://nypost.com/2026/01/10/us-news/ilhan-om...
168,30,2026-01-10-15-31-39 +0000,nypost,Russian attacks devastate Ukraine’s power grid...,https://nypost.com/2026/01/10/world-news/ukrai...
109,29,2026-01-10-22-10-13 +0000,nypost,Thousands pack lefty activist group Monarca’s ...,https://nypost.com/2026/01/10/us-news/minnesot...
171,29,2026-01-10-14-53-33 +0000,nypost,Pain in the butt! Canadian sex shop gets lette...,https://nypost.com/2026/01/10/us-news/canadian...
156,27,2026-01-10-17-00-00 +0000,wsj,In an industry town that has grown increasingl...,https://www.wsj.com/politics/policy/save-calif...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
